In this code we have used UCF50 data set, where i've considered only the 5 classes from it(because of the space sortage).

# 1. Importing the Libraries



In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf
from tensorflow import keras 
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
#import keras_metrics as km
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
import pickle
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%pwd

In [3]:
import shutil # To perform the file operation 
data_dir = '../input/ucf50/UCF50' # Original input data directory
os.mkdir('/kaggle/working/UCF50')  # Making my working directory with 5 classes
base_dir = '/kaggle/working/UCF50'
classes = ["BaseballPitch", "CleanAndJerk", "Diving", "Fencing","HighJump"]

"""Copying the 5 classes of the dataset into my working directory."""
for i in classes:
    path = os.path.join(data_dir, i)
    _, dirs, files = next(os.walk(path))
    #print(files)
    for j in files:
        src = os.path.join(path, j)
        dst = os.path.join(base_dir, i)
        
        if not os.path.exists(dst):
            os.mkdir(dst)
            
        shutil.copy(src, dst) 

In [4]:
data_dir = '/kaggle/working/UCF50'

# Shape of the image 
img_height , img_width = 64, 64 

# sequencec length of the video frame list
seq_len = 70 

print(len(classes)) 
print(classes)

In [5]:
#  Creating frames from videos
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
    vidObj.release()
    
    return frames_list

The Above code is to create the data frames from the video using the cv2 library.
>  Videobj = cv2.VideoCapture(video_path)
> > val, image =  Videobj.read()

This will read the video and creates a frame and gives value in binary format, True or False. If the value i strue then we are adding that frame into the frame list of length 70.

In [6]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [7]:
# Test, train directory 
working_dir = '/kaggle/working'
data = os.path.join(working_dir, 'X_data')
label = os.path.join(working_dir, 'Y_label')
#os.mkdir(data)
#os.mkdir(label)
X_data, Y_label = create_data(data_dir)

In [8]:
print("{} : X size, {} : y size  ".format(len(X_data), len(Y_label)) )

#print("{} : X\n\n{} : Y\n".format(X_data, Y_label))

In [9]:
# Model Creation
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(5, activation = "softmax"))

In [10]:
# X = os.listdir('/kaggle/working/X_data')
# Y =  os.listdir('/kaggle/working/Y_label')
# #print(X)
# print("{} : Length of X\n{} : Length of Y\n".format(len(X), len(Y)))

In [62]:

X_train, X_test, y_train, y_test = train_test_split( X_data, Y_label , test_size=0.20, shuffle=True, random_state=0)

In [12]:
print(" {} : Length of X_train\n {} : Length of Y_train\n {} : Length of X_test\n {} : Length of Y_test\n ".format(len(X_train), len(y_train), len(X_test), len(y_test)))

#print(" {} : X_train\n\n {} : Y_train\n\n {} : X_test\n\n {} : Y_test\n\n ".format(X_train, y_train, X_test, y_test))



In [13]:
model.summary()

In [14]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [15]:
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

In [16]:
history = model.fit( X_train, y_train, epochs=40, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)

with open('history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)


In [55]:
y_pred = model.predict(X_test)


In [30]:
# Save the trained model 
# with open('model_pkl', 'wb') as files:
#     pickle.dump(model, files)
# #ConvLSTM = pickle.dump(history)

# #model = pickle.load(ConvLSTM)

# save the model to disk
# import joblib

# filename = 'model.sav'
# joblib.dump(model, filename)

In [57]:
print(y_pred)


In [65]:
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)	
# #y_pred = model.predict(X_test)

In [66]:
print(y_pred)

In [67]:
print(y_test)

In [69]:
from sklearn.metrics import classification_report
import pandas as pd
#print(classification_report(y_test, y_pred))

report_dict = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report_dict)